# Aim

1. Make featurized data model ready
1. flatten array features e.g kw_vec (keyword vectors)

In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
DATA_ROOT = f"../data"

In [30]:
df_train = pd.read_pickle(f"{DATA_ROOT}/train/featurized/data.pkl")

In [3]:
df_test = pd.read_pickle(f"{DATA_ROOT}/test/featurized/data.pkl")

In [31]:
def prepare_model_data(df):

    kw_list = [
        "kw_" + i[0] for i in pd.read_pickle(f"{DATA_ROOT}/train/keywords.pkl")[:15]
    ]

    df_kw = pd.DataFrame(df["kw_vec"].tolist(), columns=kw_list)
    df_final = pd.concat([df_kw, df], axis="columns").drop(columns=["ID", "kw_vec"])
    return df_final


df_model_train = prepare_model_data(df_train)
df_model_test = prepare_model_data(df_test)

As we are preparing data for tree based models, there is no need to scale/ standardize the data.

In [32]:
ordered_feature_list = [
    # engineered features
    "kw_Accident",
    "kw_Northbound",
    "kw_Hwy",
    "kw_ramp",
    "kw_slow",
    "kw_Trl",
    "kw_Mopac",
    "kw_Okeechobee",
    "kw_Brookshire",
    "kw_Huntington",
    "kw_NYS",
    "kw_Fuqua",
    "kw_Middlefield",
    "kw_JFK",
    "kw_Cedarhurst",
    "zip_02",
    "zip_25",
    "zip_len",
    "zip_is_compound",
    # raw features
    "Source",
    "TMC",
    "Distance(mi)",
    "Side",
    "City",
    "County",
    "State",
    "Timezone",
    "Airport_Code",
    "Temperature(F)",
    "Humidity(%)",
    "Pressure(in)",
    "Visibility(mi)",
    "Wind_Direction",
    "Wind_Speed(mph)",
    "Weather_Condition",
    "Amenity",
    "Bump",
    "Crossing",
    "Give_Way",
    "Junction",
    "No_Exit",
    "Railway",
    "Roundabout",
    "Station",
    "Stop",
    "Traffic_Calming",
    "Traffic_Signal",
    "Turning_Loop",
    "Sunrise_Sunset",
    "Civil_Twilight",
    "Nautical_Twilight",
    "Astronomical_Twilight",
    # predict
    "Severity",
]

In [34]:
os.makedirs(f"{DATA_ROOT}/train/model/", exist_ok=True)
os.makedirs(f"{DATA_ROOT}/test/model/", exist_ok=True)

In [37]:
df_model_train[ordered_feature_list].to_pickle(f"{DATA_ROOT}/train/model/data.pkl")
df_model_test[ordered_feature_list].to_pickle(f"{DATA_ROOT}/test/model/data.pkl")